# Humanoid Env

In [1]:
import gymnasium as gym
import numpy as np

In [2]:
# Crear el entorno Humanoid
env = gym.make("Humanoid-v4")

## Q-learning

In [ ]:
# Inicializar la tabla Q con valores iniciales
action_space_size = env.action_space.n
state_space_size = env.observation_space.shape[0]
q_table = np.zeros((state_space_size, action_space_size))

In [ ]:
# Definir parámetros
alpha = 0.1  # Tasa de aprendizaje
gamma = 0.99  # Factor de descuento
epsilon = 0.1  # Probabilidad de exploración inicial
num_episodes = 1000